In [ ]:
!pip install -q pyclipper
!pip install -q Polygon3

     |████████████████████████████████| 603 kB 5.0 MB/s 


In [ ]:
!cp "./drive/MyDrive/shared/zerowaste-f-final.zip" ./
!unzip -q "./zerowaste-f-final.zip"

import os
os.mkdir("./images")
os.mkdir("./labels")

In [ ]:
import os
import numpy as np
import cv2
import json
import random
import matplotlib.pyplot as plt
import Polygon as plg
import pyclipper
import pickle

from PIL import Image


# set plot size
def set_plot_size(name):
    # (w, h)
    if name == "wide":
        plt.rcParams["figure.figsize"] = (30, 4)
        return
    if name == "big":
        plt.rcParams["figure.figsize"] = (30, 10)  # (w, h)
        return
    plt.rcParams["figure.figsize"] = (6, 4)  # normal

In [ ]:
# ----- #
# get data_dict_all
# ----- #
def json2dict(path):
    with open(path) as f:
        return json.load(f)
     

labels_test = json2dict("./splits_final_deblurred/test/labels.json")

img_all = labels_test["images"]
anno_raw = labels_test["annotations"]
anno_all = [list() for idx in range(929)]

for idx in range(len(anno_raw)):
    tmp = anno_raw[idx]
    idx_img = tmp["image_id"]
    anno_all[idx_img].append(tmp)


def get_zerowaste_dicts():
    dataset_dicts = []
    for idx0 in range(len(img_all)):
        record = dict()
        tmp = img_all[idx0]
        record["file_name"] = os.path.join("./splits_final_deblurred/test/data", tmp["file_name"])
        record["image_id"] = tmp["id"]
        record["width"] = tmp["width"]
        record["height"] = tmp["height"]
        
        tmp_tmp = anno_all[idx0]
        objs = list()
        for item in tmp_tmp:
            obj = dict()
            obj["bbox"] = item["bbox"]
            obj["bbox_mode"] = "XYWH_ABS"
            obj["segmentation"] = item["segmentation"]
            obj["category_id"] = item["category_id"] - 1
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts


data_dict_all = get_zerowaste_dicts()

In [ ]:
for idx0 in range(929):  # 929
    name = str(idx0).zfill(5)

    data_dict = data_dict_all[idx0]
    path_img = data_dict["file_name"]
    Image.open(path_img).resize((910, 512)).save("./images/" + name + ".jpg")

    anno_all = data_dict["annotations"]
    out = []
    for idx1 in range(len(anno_all)):
        anno = anno_all[idx1]
        out.append((anno["segmentation"], anno["category_id"]))
        
    with open("./labels/" + name + ".pkl", 'wb') as f:
        pickle.dump(out, f)


    if (idx0 + 1) % 100 == 0:
        print(idx0)


99
199
299
399
499
599
699
799
899


In [ ]:
!zip -rq "./images_test.zip" "./images"
!zip -rq "./labels_test.zip" "./labels"

!cp "./images_test.zip" "./drive/MyDrive/cs523_project/"
!cp "./labels_test.zip" "./drive/MyDrive/cs523_project/"